In [2]:
import hashlib
import os
import tarfile
import zipfile
import requests

import numpy as np
import pandas as pd
import torch
from torch import nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = 'cpu'

In [3]:
DATA_HUB = dict()
DATA_URL = 'http://d2l-data.s3-accelerate.amazonaws.com/'

def download(name, cache_dir=os.path.join('.', 'data')):
    """下载一个DATA_HUB中的文件，返回本地文件名"""
    assert name in DATA_HUB, f"{name} 不存在于{DATA_HUB}"
    url, sha1_hash = DATA_HUB[name]
    os.makedirs(cache_dir, exist_ok=True)
    fname = os.path.join(cache_dir, url.split('/')[-1])
    if os.path.exists(fname):
        sha1 = hashlib.sha1()
        with open(fname, 'rb') as f:
            while True:
                data = f.read(1048576) # 读取1MB
                if not data:
                    break
                sha1.update(data)
        if sha1.hexdigest() == sha1_hash:
            return fname # 命中缓存
    
    print(f'正在从{url}下载{fname}...')
    r = requests.get(url, stream=True, verify=True)
    with open(fname, 'wb') as f:
        f.write(r.content)
    return fname

def dowload_extract(name, folder=None):
    """下载并解压zip/tar文件"""
    fname = download(name)
    base_dir = os.path.dirname(fname)
    data_dir, ext = os.path.splitext(name)
    if ext == 'zip':
        fp = zipfile.ZipFile(fname, 'r')
    elif ext in ('.tar', '.gz'):
        fp = tarfile.open(fname, 'r')
    else:
        assert False
    fp.extractall(base_dir)
    return os.path.join(base_dir, folder) if folder else data_dir
    
def download_all():
    """下载DATA_HUB中所有的文件"""
    for name in DATA_HUB:
        download(name)

In [4]:
DATA_HUB['kaggle_house_train'] = (
    DATA_URL + 'kaggle_house_pred_train.csv',
    '585e9cc93e70b39160e7921475f9bcd7d31219ce')

DATA_HUB['kaggle_house_test'] = (
    DATA_URL + 'kaggle_house_pred_test.csv',
    'fa19780a7b011d9b009e8bff8e99922a8ee2eb90')

In [5]:
train_data = pd.read_csv(download('kaggle_house_train'))
test_data = pd.read_csv(download('kaggle_house_test'))

In [6]:
print(train_data.shape)
print(test_data.shape)

# 输出前四个和最后两个特征，以及对应的标签
print(train_data.iloc[0:4, [0, 1, 2, 3, -3, -2, -1]])

train_labels, train_input = train_data['SalePrice'].copy(), train_data.drop('SalePrice', axis=1)
test_input = test_data

print(train_input.info(), train_labels.info(), test_input.info())

(1460, 81)
(1459, 80)
   Id  MSSubClass MSZoning  LotFrontage SaleType SaleCondition  SalePrice
0   1          60       RL         65.0       WD        Normal     208500
1   2          20       RL         80.0       WD        Normal     181500
2   3          60       RL         68.0       WD        Normal     223500
3   4          70       RL         60.0       WD       Abnorml     140000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities   

In [7]:
# 补全不完全的特征，将特征缩放到零均值和单位方差来标准化数据
# 标准化原因：
# 1、方便优化
# 2、对所有特征的惩罚都相同

def data_preprocess(all_features):
    # 得到所有数值类型（非文本）的索引，all_features.dtypes为各列数据类型
    numeric_features = all_features.dtypes[all_features.dtypes != 'object'].index
    all_features[numeric_features] = all_features[numeric_features].apply(
                        lambda x: (x - x.mean()) / (x.std()))

    # 标准化数据后，均值消失（变为0），可以将缺失值设置为0（即均值）
    all_features[numeric_features] = all_features[numeric_features].fillna(0)
    
    # get_dummies为所有非数值变量进行独热编码，dummy_na=True表示非数值变量中的NA也会生成一列独热编码
    all_features = pd.get_dummies(all_features, dummy_na=True)
    return all_features

In [8]:
all_features = pd.concat((train_input.iloc[:, 1:], test_input.iloc[:, 1:]))
all_features_ = data_preprocess(all_features).astype(float)

In [9]:
# 转换为tensor|
# with pd.option_context('display.max_columns', None, 'display.max_rows', None):
#     print(train_input_.iloc[:, 1:].dtypes)

n_train = train_input.shape[0]
train_features = torch.tensor(all_features_[:n_train].to_numpy(), dtype=torch.float32)
test_features = torch.tensor(all_features_[n_train:].to_numpy(), dtype=torch.float32)
train_labels_ = torch.tensor(train_labels.to_numpy().reshape(-1, 1), dtype=torch.float32)

print(train_features.shape, test_features.shape, train_labels.shape)

torch.Size([1460, 330]) torch.Size([1459, 330]) (1460,)


In [10]:
# 建立简单模型（这里用线性模型）作为baseline，判断其他模型的性能比简单模型优越多少
loss = nn.MSELoss()
input_num = train_features.shape[1]
print(input_num)

def get_net():
    net = nn.Sequential(
        nn.Linear(input_num, 1),
    ).to(device)
    return net

330


In [11]:
# 对于房价来说，我们更关心相对误差而非绝对误差
# 对于12万元的房价和2000万元的房价，预测偏差10万元，产生的影响并不相同

# 使用对数值的均方根误差
def log_rmse(net, features, labels):
    # 防止对数小于1下溢，将小于1的值设为1
    clipped_preds = torch.clamp(net(features), 1, float('inf'))
    
    rmse = torch.sqrt(loss(torch.log(clipped_preds), torch.log(labels)))
    return rmse.item()

In [12]:
from utils.fashion_mnist import load_array
# 使用Adam优化器，Adam优化器的主要有点在于对初始学习率并不敏感
def train(net, train_features, train_labels, test_features, test_labels,
         num_epochs, learning_rate, weight_decay, batch_size):
    train_ls, test_ls = [], []
    train_iter = load_array((train_features, train_labels), batch_size)
    
    optimizer = torch.optim.Adam(net.parameters(), 
                                 lr=learning_rate,
                                 weight_decay=weight_decay)
    
    for epoch in range(num_epochs):
        for X, y in train_iter:
            X, y = X.to(device), y.to(device)
            
            optimizer.zero_grad()
            l = loss(net(X), y)
            l.backward()
            optimizer.step()
        
        train_ls.append(log_rmse(net, train_features.to(device), train_labels.to(device)))
        if test_labels is not None:
            test_ls.append(log_rmse(net, test_features.to(device), test_labels.to(device)))
            
    
    return train_ls, test_ls

In [13]:
# K折交叉验证
# 选择第i个切片作为验证数据，其余部分作为训练数据
def get_k_fold_data(k, i, X, y):
    assert k > 1
    
    fold_size = X.shape[0] // k
    X_train, y_train = None, None
    for j in range(k):
        idx = slice(j * fold_size, (j + 1) * fold_size)
        X_part, y_part = X[idx, :], y[idx]
        
        if j == i:
            X_valid, y_valid = X_part, y_part
        elif X_train is None:
            X_train, y_train = X_part, y_part
        else:
            X_train = torch.cat([X_train, X_part], 0)
            y_train = torch.cat([y_train, y_part], 0)
    return X_train, y_train, X_valid, y_valid

In [14]:
# 在K折交叉验证中训练K次
def k_fold(k, X_train, y_train, num_epochs, learning_rate, weight_decay, batch_size):
    train_l_sum, valid_l_sum = 0, 0
    for i in range(k):
        data = get_k_fold_data(k, i, X_train, y_train)
        net = get_net()
        train_ls, valid_ls = train(net, *data, num_epochs, learning_rate, 
                                   weight_decay, batch_size)
        train_l_sum += train_ls[-1]
        valid_l_sum += valid_ls[-1]
        
        print(f'折{i + 1}，训练log rmse{float(train_ls[-1]):f}, '
              f'验证log rmse{float(valid_ls[-1]):f}')
    
    return train_l_sum / k, valid_l_sum / k

In [15]:
k, num_epochs, lr, weight_decay, batch_size = 5, 300, 5, 0, 64
train_l, valid_l = k_fold(k, train_features, train_labels_, num_epochs, lr,
            weight_decay, batch_size)
print(f'{k}-折验证: 平均训练log rmse: {float(train_l):f}, '
      f'平均验证log rmse: {float(valid_l):f}')

折1，训练log rmse0.132256, 验证log rmse0.144748
折2，训练log rmse0.129320, 验证log rmse0.146675
折3，训练log rmse0.128025, 验证log rmse0.141958
折4，训练log rmse0.134044, 验证log rmse0.136722
折5，训练log rmse0.125818, 验证log rmse0.166770
5-折验证: 平均训练log rmse: 0.129893, 平均验证log rmse: 0.147375


In [16]:
def train_and_pred(train_features, test_features, train_labels, test_data,
                   num_epochs, lr, weight_decay, batch_size):
    net = get_net()
    train_ls, _ = train(net, train_features, train_labels, None, None,
                        num_epochs, lr, weight_decay, batch_size)
    
    print(f'训练log rmse：{float(train_ls[-1]):f}')
    # 将网络应用于测试集。
    preds = net(test_features.to(device)).detach().cpu().numpy()
    # 将其重新格式化以导出到Kaggle
    test_data['SalePrice'] = pd.Series(preds.reshape(1, -1)[0])
    submission = pd.concat([test_data['Id'], test_data['SalePrice']], axis=1)
    submission.to_csv('submission.csv', index=False)

In [17]:
train_and_pred(train_features, test_features, train_labels_, test_data,
               num_epochs, lr, weight_decay, batch_size)

训练log rmse：0.127907
